In [2]:
!pip install git+https://github.com/cheind/py-motmetrics.git

  Cloning https://github.com/cheind/py-motmetrics.git to /private/var/folders/h8/jvpg08h90c5bggxybw41glg40000gp/T/pip-req-build-pef195w5
  Running command git clone -q https://github.com/cheind/py-motmetrics.git /private/var/folders/h8/jvpg08h90c5bggxybw41glg40000gp/T/pip-req-build-pef195w5
  Resolved https://github.com/cheind/py-motmetrics.git to commit b9ca89176f4b095c875312356a68317403e19db7


In [3]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from eval_utils import *
from video_utils import *
from load_utils import *
from tracking_utils import *

data_path = '../../AICity_data/train/S03/c010/'

ModuleNotFoundError: No module named 'motmetrics'

In [ ]:
gt_detect = readDetectionsXML('ai_challenge_s03_c010-full_annotation.xml')
gt_notParked = getNotParkedCars(gt_detect)

In [ ]:
retinanet_101_detections = {}
with open('retinanet_101_detections.pkl','rb') as openFile:
    retinanet_101_detections = pickle.load(openFile)